In [3]:
import requests as r
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import os ,sys

In [ ]:
url = 'https://www.magicbricks.com/property-for-rent/residential-real-estate?proptype=Multistorey-Apartment,Builder-Floor-Apartment,Penthouse,Studio-Apartment,Service-Apartment,Residential-House,Villa&cityName=New%20Delhi'

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Use the webdriver.Chrome() constructor without the executable_path argument
driver = webdriver.Chrome()


try:
    driver.get(url)

    # Define a function to scroll to the bottom of the page to load dynamic content
    def scroll_to_bottom():
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(30)  # Adjust the waiting time as needed
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

    # Scroll to the bottom of the page to load all content
    scroll_to_bottom()

    # Extract the HTML content after the dynamic content has been loaded
    data = driver.page_source

    # Create BeautifulSoup object for parsing
    soup = BeautifulSoup(data, 'html.parser')

    # Find all the items on the page (assuming each item has the same structure)
    all_items = soup.find_all('div', attrs={'class': 'mb-srp__card'})

    # Create a list to store all the dictionaries containing details for each item
    all_details = []

    # Loop through all the items
    for item in all_items:
        title = item.find('h2', attrs={'class': 'mb-srp__card--title'})
        price = item.find('div', attrs={'class': 'mb-srp__card__price--amount'})
        details = item.find_all('div', attrs={'class': 'mb-srp__card__summary--value'})
        label = item.find_all('div', attrs={'class': 'mb-srp__card__summary--label'})

        if title and price:  # Check if title and price are present
            full_detail = {'title': title.text.strip(), 'price': price.text.strip()}
        else:
            continue  # Skip this item if title or price is missing

        for i in range(min(len(details), len(label))):
            key = label[i].text.strip()  # Use the label text as the key
            value = details[i].text.strip()  # Use the details text as the value

            if key in full_detail:  # If a key already exists, convert the value to a list
                existing_value = full_detail[key]
                if not isinstance(existing_value, list):
                    full_detail[key] = [existing_value]
                full_detail[key].append(value)
            else:
                full_detail[key] = value

        all_details.append(full_detail)

except Exception as e:
    # Handle the exception
    print(f"An error occurred: {e}")

finally:
    # Close the WebDriver in the 'finally' block to ensure it's always closed
    driver.quit()